In [2]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split #We need this to split the data
from sklearn.metrics import confusion_matrix
import sklearn as sk

Ik heb de fifa 18 dataset gekozen omdat ik het spel al jaren speel. Ik ben erg bekend met fifa player cards en hoe de prijzen en statistieken in het spel werken. Ik ben ook benieuwd wat de grootste oorzaken zijn voor prijsveranderingen.

In [5]:
df=pd.read_csv("CompleteDataset.csv")
df.head()


C:\Users\schol\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (23,35) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
0,0,Cristiano Ronaldo,32,https://cdn.sofifa.org/48/18/players/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Real Madrid CF,https://cdn.sofifa.org/24/18/teams/243.png,...,61.0,53.0,82.0,62.0,91.0,89.0,92.0,91.0,66.0,92.0
1,1,L. Messi,30,https://cdn.sofifa.org/48/18/players/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,93,93,FC Barcelona,https://cdn.sofifa.org/24/18/teams/241.png,...,57.0,45.0,84.0,59.0,92.0,90.0,88.0,91.0,62.0,88.0
2,2,Neymar,25,https://cdn.sofifa.org/48/18/players/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,94,Paris Saint-Germain,https://cdn.sofifa.org/24/18/teams/73.png,...,59.0,46.0,79.0,59.0,88.0,87.0,84.0,89.0,64.0,84.0
3,3,L. Suárez,30,https://cdn.sofifa.org/48/18/players/176580.png,Uruguay,https://cdn.sofifa.org/flags/60.png,92,92,FC Barcelona,https://cdn.sofifa.org/24/18/teams/241.png,...,64.0,58.0,80.0,65.0,88.0,85.0,88.0,87.0,68.0,88.0
4,4,M. Neuer,31,https://cdn.sofifa.org/48/18/players/167495.png,Germany,https://cdn.sofifa.org/flags/21.png,92,92,FC Bayern Munich,https://cdn.sofifa.org/24/18/teams/21.png,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


De reden dat ik hieronder ook df.info doe is omdat ik zichtbaar moet hebben welke kolommen er zijn en ook of dat integers, floats of strings zijn. Ook kan ik hierdoor veel beter data cleanen.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17981 entries, 0 to 17980
Data columns (total 75 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           17981 non-null  int64  
 1   Name                 17981 non-null  object 
 2   Age                  17981 non-null  int64  
 3   Photo                17981 non-null  object 
 4   Nationality          17981 non-null  object 
 5   Flag                 17981 non-null  object 
 6   Overall              17981 non-null  int64  
 7   Potential            17981 non-null  int64  
 8   Club                 17733 non-null  object 
 9   Club Logo            17981 non-null  object 
 10  Value                17981 non-null  object 
 11  Wage                 17981 non-null  object 
 12  Special              17981 non-null  int64  
 13  Acceleration         17981 non-null  object 
 14  Aggression           17981 non-null  object 
 15  Agility              17981 non-null 

Hieronder staat in Markdown welke kolommen ik eigenlijk zou willen gebruiken om eigenschappen te voorspellen. Deze omdat de statistieken vaak bepalen wat de waarde is van een speler en ook wat de ratings zijn. Posities spelen ook een rol maar veel kleiner dan speler statistieken.

"Acceleration","Aggression", "Agility", "Balance",
"Ball control", 'Composure', 'Crossing', 'Curve', 'Dribbling', 'Finishing', 'Free kick accuracy', 'GK diving', 'GK handling', 'GK kicking', 'GK positioning',
'GK reflexes', 'Heading accuracy', 'Interceptions', 'Jumping', 'Long passing', 'Long shots', 'Marking', 'Penalties', 'Positioning', 'Reactions', 'Short passing',
'Shot power', 'Sliding tackle', 'Sprint speed', 'Stamina', 'Standing tackle','Strength', 'Vision', 'Volleys']]

Uiteindelijk heb ik deze kolommen in de dataset gebruikt. Dit alleen omdat dit integers en floats zijn. Daardoor valt er ook mee te rekenen. Het is ook niet mogelijk om dummies te creeeren omdat verreweg de meeste data in deze dataset niet categoriaal is. Echter is dit wel nodig om dummies te creeeren. Ik weet niet of het mogelijk is om strings te veranderen naar floats en integers in dit geval maar ik weet niet hoe in ieder geval.

In [52]:
df_select=df[["Age", "Overall","Special", 'CAM', "CB", "CDM", "CF", "CM", "ID", "LAM", "LB", "LCB", "LCM", "LDM", "LF", "LM", "LS", "LW", "LWB", "RAM", "RB",
"RCB","RCM","RDM","RF","RM","RS","RW","RWB","ST"]]
df_select.head()

,Age,Overall,Special,CAM,CB,CDM,CF,CM,ID,LAM,...,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
0,32,94,2228,89.0,53.0,62.0,91.0,82.0,20801,89.0,...,61.0,53.0,82.0,62.0,91.0,89.0,92.0,91.0,66.0,92.0
1,30,93,2154,92.0,45.0,59.0,92.0,84.0,158023,92.0,...,57.0,45.0,84.0,59.0,92.0,90.0,88.0,91.0,62.0,88.0
2,25,92,2100,88.0,46.0,59.0,88.0,79.0,190871,88.0,...,59.0,46.0,79.0,59.0,88.0,87.0,84.0,89.0,64.0,84.0
3,30,92,2291,87.0,58.0,65.0,88.0,80.0,176580,87.0,...,64.0,58.0,80.0,65.0,88.0,85.0,88.0,87.0,68.0,88.0
5,28,91,2143,84.0,57.0,62.0,87.0,78.0,188545,84.0,...,58.0,57.0,78.0,62.0,87.0,82.0,88.0,84.0,61.0,88.0


In [41]:
df = df.dropna()

Hieronder gebruik ik die drie kolommen omdat dit alle 3 integers zijn die een hele belangrijke eigenschap van een speler bevat. De balans tussen deze 3 onderdelen is erg interessant. Hier kan je het best zien welk type statistieken de meeste invloed op elkaar hebben.

In [ ]:
sns.pairplot(df_select[['Overall','Special',"Age",]])
plt.show()

Ik gebruik in weekopdracht 5 het KNN model om variabelen te voorspellen.Dit model categoriseerd variabelen door te kijken welke andere regel binnen bepaalde variabelen het meest dezelfde eigenschappen hebben. Vervolgens worden deze regels geclusterd bij vergelijkbare regels waardoor het mogelijk is om te voorspellen waar een nieuwe regel hoort. Dit omdat alle regels geclusterd zijn bij vegrelijkbare regels.

Hieronder gebruik ik voor de x waardes ik wil voorspellen age en overall. Hiermee wil ik de y waarde special voorspellen. Ik zou deze intitieel voorspellen met veel meer x variabelen. Echter bleek de succesgraad van voospelling extreem laag te zijn met de variabelen die ik tot mijn beschikking had. Daarom heb ik alleen deze 3 hier gebruikt. 

# Weekopdracht 5

In [69]:
from sklearn.preprocessing import normalize

X = df_select.loc[:,'Age':'Overall'] #create the X matrix
y = df_select['Special'] #create the y-variable

X = normalize(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
pd.DataFrame(X_train).head()

,0,1
0,0.332820,0.942990
1,0.333421,0.942778
2,0.321582,0.946882
3,0.400819,0.916157
4,0.478852,0.877896


In [70]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier() 
knn = knn.fit(X_train, y_train)

De score hieronder is extreem slecht maar ik heb het met meerdere x/variabelen geprobeerd maar heb geen een keer een goede score gekregen

In [71]:
knn.score(X_test, y_test)

0.0029635901778154107

Hieronder geef ik met een classification report de precision, recall en f1/score en support weer met behulp van 1 functie.

X =df_select.loc[:,'Age':'Overall']  #This gets all the columns from Time to Amount
y = df_select['Special']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=1, n_estimators=100) #RF is a random algorithm, so to get the same results we need to use random_state
rf = rf.fit(X_train, y_train)

rf.score(X_test,y_test)

In [68]:
from sklearn.metrics import classification_report

for i in range(1,11):
    knn = KNeighborsClassifier(n_neighbors = i) #make a new kNN model with i (1-10) neighbors
    knn = knn.fit(X_train, y_train) #fit new model on train data
    y_test_pred = knn.predict(X_test) #predict using new model, with test data
    print(f"With {i} neighbors the result is:")
    print(classification_report(y_test, y_test_pred_new)) #use a built-in function to print out accuracy, precision and recall

With 1 neighbors the result is:
              precision    recall  f1-score   support

        1025       0.00      0.00      0.00         0
        1031       0.00      0.00      0.00         0
        1057       0.00      0.00      0.00         1
        1085       0.00      0.00      0.00         0
        1110       0.00      0.00      0.00         0
        1113       0.00      0.00      0.00         0
        1120       0.00      0.00      0.00         0
        1121       0.00      0.00      0.00         0
        1128       0.00      0.00      0.00         0
        1130       0.00      0.00      0.00         0
        1131       0.00      0.00      0.00         1
        1132       0.00      0.00      0.00         0
        1137       0.00      0.00      0.00         0
        1145       0.00      0.00      0.00         1
        1146       0.00      0.00      0.00         1
        1149       0.00      0.00      0.00         1
        1150       0.00      0.00      0.00      

C:\Users\schol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\schol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\schol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\schol\anaconda3\lib\site-packages\sklearn\metrics\_cla

With 2 neighbors the result is:
              precision    recall  f1-score   support

        1025       0.00      0.00      0.00         0
        1031       0.00      0.00      0.00         0
        1057       0.00      0.00      0.00         1
        1085       0.00      0.00      0.00         0
        1110       0.00      0.00      0.00         0
        1113       0.00      0.00      0.00         0
        1120       0.00      0.00      0.00         0
        1121       0.00      0.00      0.00         0
        1128       0.00      0.00      0.00         0
        1130       0.00      0.00      0.00         0
        1131       0.00      0.00      0.00         1
        1132       0.00      0.00      0.00         0
        1137       0.00      0.00      0.00         0
        1145       0.00      0.00      0.00         1
        1146       0.00      0.00      0.00         1
        1149       0.00      0.00      0.00         1
        1150       0.00      0.00      0.00      

C:\Users\schol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\schol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\schol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\schol\anaconda3\lib\site-packages\sklearn\metrics\_cla

With 3 neighbors the result is:
              precision    recall  f1-score   support

        1025       0.00      0.00      0.00         0
        1031       0.00      0.00      0.00         0
        1057       0.00      0.00      0.00         1
        1085       0.00      0.00      0.00         0
        1110       0.00      0.00      0.00         0
        1113       0.00      0.00      0.00         0
        1120       0.00      0.00      0.00         0
        1121       0.00      0.00      0.00         0
        1128       0.00      0.00      0.00         0
        1130       0.00      0.00      0.00         0
        1131       0.00      0.00      0.00         1
        1132       0.00      0.00      0.00         0
        1137       0.00      0.00      0.00         0
        1145       0.00      0.00      0.00         1
        1146       0.00      0.00      0.00         1
        1149       0.00      0.00      0.00         1
        1150       0.00      0.00      0.00      

...

C:\Users\schol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\schol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\schol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\schol\anaconda3\lib\site-packages\sklearn\metrics\_cla

With 9 neighbors the result is:
              precision    recall  f1-score   support

        1025       0.00      0.00      0.00         0
        1031       0.00      0.00      0.00         0
        1057       0.00      0.00      0.00         1
        1085       0.00      0.00      0.00         0
        1110       0.00      0.00      0.00         0
        1113       0.00      0.00      0.00         0
        1120       0.00      0.00      0.00         0
        1121       0.00      0.00      0.00         0
        1128       0.00      0.00      0.00         0
        1130       0.00      0.00      0.00         0
        1131       0.00      0.00      0.00         1
        1132       0.00      0.00      0.00         0
        1137       0.00      0.00      0.00         0
        1145       0.00      0.00      0.00         1
        1146       0.00      0.00      0.00         1
        1149       0.00      0.00      0.00         1
        1150       0.00      0.00      0.00      

C:\Users\schol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\schol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\schol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\schol\anaconda3\lib\site-packages\sklearn\metrics\_cla

With 10 neighbors the result is:
              precision    recall  f1-score   support

        1025       0.00      0.00      0.00         0
        1031       0.00      0.00      0.00         0
        1057       0.00      0.00      0.00         1
        1085       0.00      0.00      0.00         0
        1110       0.00      0.00      0.00         0
        1113       0.00      0.00      0.00         0
        1120       0.00      0.00      0.00         0
        1121       0.00      0.00      0.00         0
        1128       0.00      0.00      0.00         0
        1130       0.00      0.00      0.00         0
        1131       0.00      0.00      0.00         1
        1132       0.00      0.00      0.00         0
        1137       0.00      0.00      0.00         0
        1145       0.00      0.00      0.00         1
        1146       0.00      0.00      0.00         1
        1149       0.00      0.00      0.00         1
        1150       0.00      0.00      0.00     

C:\Users\schol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\schol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\schol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\schol\anaconda3\lib\site-packages\sklearn\metrics\_cla